In [94]:
!pip install wandb
import wandb
# Replace with your actual API key
api_key = "17dab9d1bbdc37c41831799a4b0b50d3e97400c5"
project_name='RBCS2'
# Login to Weights & Biases
wandb.login(key=api_key)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [95]:
import gym
import random
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 10)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
torch.manual_seed(0)
import base64
import io  # For visualization
from gym.wrappers.monitoring import video_recorder
from IPython.display import HTML
from IPython import display
import glob
import datetime

In [96]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

In [97]:
env = gym.make('CartPole-v1')
env.seed(0)
state_shape = env.observation_space.shape[0]
action_shape = env.action_space.n
# print('observation space:', env.observation_space)
# print('action space:', env.action_space)


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


In [98]:
class Policy(nn.Module):
    def __init__(self, state_size=4, action_size=2, hidden_size=32):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, action_size)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = self.fc2(x)
        # we just consider 1 dimensional probability of action
        return F.softmax(x, dim=1)

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        if state.shape[1] != 4:  # Check if the state tensor has the expected shape
            state = state.view(1, 4)  # Reshape the state tensor to (1, 4)
        probs = self.forward(state).cpu()
        model = Categorical(probs)
        action = model.sample()
        return action.item(), model.log_prob(action)

In [99]:
class Baseline(nn.Module):
    def __init__(self, state_size, hidden_size=32):
        super(Baseline, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = self.fc2(x)
        return x

In [100]:
def reinforce(env, policy, optimizer, baseline, baseline_optimizer, n_episodes=1000, max_t=1000, gamma=1.0, print_every=100):
    scores_window = deque(maxlen=100)
    scores = []
    for e in range(1, n_episodes):
        saved_log_probs = []
        rewards = []
        states = []
        state = env.reset()
        average_scores_epsgrdy = []
        episode_list_epsgrdy = []
        for t in range(max_t):
            states.append(state)
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break
        scores_window.append(sum(rewards))
        scores.append(sum(rewards))

        average_score = np.mean(scores_window)
        average_scores_epsgrdy.append(average_score)

        wandb.log({'average_score': average_score})

        episode_list_epsgrdy.append(e)


        discounts = [gamma ** i for i in range(len(rewards) + 1)]
        R = sum([a * b for a, b in zip(discounts, rewards)])

        # Calculate the baseline
        baseline_values = []
        for state_in_trajectory in states:
            state_tensor = torch.from_numpy(state_in_trajectory).float().unsqueeze(0).to(device)
            baseline_value = baseline(state_tensor)
            baseline_values.append(baseline_value)
        baseline_values = torch.cat(baseline_values, dim=0).requires_grad_()

        # Calculate the policy loss
        policy_loss = []
        for log_prob, baseline_value in zip(saved_log_probs, baseline_values):
            policy_loss.append(-(log_prob * (R - baseline_value)))
        policy_loss = torch.cat(policy_loss).sum()

        # Update the policy
        optimizer.zero_grad()
        policy_loss.backward(retain_graph=True)  # Retain the computation graph
        optimizer.step()

        # Update the baseline
        baseline_optimizer.zero_grad()
        baseline_loss = ((baseline_values - R) ** 2).mean()
        baseline_loss.backward()
        baseline_optimizer.step()

        if e % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(e, np.mean(scores_window)))
        if np.mean(scores_window) >= 195.0:
            print(f'Environment {env.unwrapped.spec.id} solved in {e - 100:d} episodes!\tAverage Score: {np.mean(scores_window):.2f}')
            break

    return episode_list_epsgrdy, average_scores_epsgrdy


# policy = Policy().to(device)
# optimizer = optim.Adam(policy.parameters(), lr=1e-2)
# baseline = Baseline(state_size=env.observation_space.shape[0]).to(device)
# baseline_optimizer = optim.Adam(baseline.parameters(), lr=1e-2)
# scores = reinforce(env, policy, optimizer, baseline, baseline_optimizer, n_episodes=2000)
# wandb.finish()  # Finish wandb run

In [101]:
wandb.init(project=project_name)

In [102]:
# env = gym.make('CartPole-v1')
# env.seed(0)
# state_shape = env.observation_space.shape[0]
# action_shape = env.action_space.n

In [103]:
sweep_config = {
    'method': 'grid',
    'metric': {
      'name': 'Average Reward',
      'goal': 'maximize'
    },
    'parameters': {
        'state_size': {
            'values': [state_shape]
        },
        'action_size': {
            'values': [action_shape]
        },
        'hidden_size': {
            'values': [64]
        },
        'max_t': {
            'values': [1000]
        },
        'lr': {
            'values': [1e-3]
        },
        'gamma': {
            'values': [0.99]
        },
        'alpha': {
            'values': [0.0001]
        },
        'n_episodes': {
            'values': [2000]
        },
        "max_t":{
            'values': [1000]
        },
         "print_every": {
             'values': [100]
         }
    }
}

In [104]:
# Initialize a new sweep
sweep_id = wandb.sweep(sweep=sweep_config,  project=project_name)


Create sweep with ID: 4752uu5s
Sweep URL: https://wandb.ai/iitm_aero/RBCS2/sweeps/4752uu5s


In [105]:
average_scores = []

In [106]:
def AvgOverExperiments():

    # with wandb.init() as run:
        # Get the hyperparameters for this run
        # config = wandb.config

        # begin_time = datetime.datetime.now()
        config= {
        'state_size': state_shape,
        'action_size':action_shape,
        'hidden_size':64,
        'max_t':1000,
        'baseline_lr':1e-3,
        'lr':1e-3,
        'gamma':0.99,
        'alpha':0.0001,
        'n_episodes':2000,
        "max_t": 1000,
        "print_every": 100
        }
        # config = sweep_config
        begin_time = datetime.datetime.now()
        # Create the policy network with the specified hyperparameters
        policy = Policy(state_size=4, action_size=2, hidden_size=config["hidden_size"])
        optimizer = optim.Adam(policy.parameters(), lr=config["lr"])


        # Create the baseline network with the specified hyperparameters
        baseline = Baseline(state_size=state_shape).to(device)
        baseline_optimizer = optim.Adam(baseline.parameters(), lr=config["baseline_lr"])


        # Run the REINFORCE algorithm with the specified hyperparameters
        episode_list_epsgrdy, average_scores_epsgrdy = reinforce(env, policy, optimizer, baseline, baseline_optimizer, n_episodes=config["n_episodes"], max_t=config["max_t"], gamma=config["gamma"], print_every=config["print_every"])

        time_taken = datetime.datetime.now() - begin_time
        print(time_taken)
        # time_taken = datetime.datetime.now() - begin_time
        print(average_scores_epsgrdy)
        print('\n')
        average_scores.append(average_scores_epsgrdy)
        # print(time_taken)
        # Log the final average score to wandb
        # wandb.log({"Average Score": average_scores_epsgrdy})
        # wandb.log({"Average Score": cumulative_regret})


In [107]:
def plot_avg_curves(avg_reward,reward_std, step_size = 1 ):

    trunc_avg_reward = avg_reward[0::step_size]
    tunc_reward_std = reward_std[0::step_size]

    # trunc_avg_step = avg_step[0::step_size]
    # tunc_step_std = step_std[0::step_size]

    # Plot the average rewards with mean and standard deviation
    plt.figure()
    plt.plot(trunc_avg_reward, label='Mean')
    plt.fill_between(range(step_size), trunc_avg_reward + tunc_reward_std, trunc_avg_reward - tunc_reward_std, alpha=0.5, label=' Std Dev')
    plt.xlabel('Episode')
    plt.ylabel('Average Total Reward')
    plt.title('Average Total Reward vs Episode')
    plt.legend()
    # plt.savefig( "RvsE " +".pdf")
    # files.download(exp_no+"_RvsE.png")
    plt.show()



    # # Plot the average rewards with mean and standard deviation
    # plt.figure()
    # plt.plot(trunc_avg_step, label='Mean')
    # plt.fill_between(range(step_size), trunc_avg_step + tunc_step_std, trunc_avg_step - tunc_step_std, alpha=0.5, label=' Std Dev')
    # plt.xlabel('Episode')
    # plt.ylabel('Average Regret')
    # plt.title(' Average Regret vs Episode')
    # plt.legend()
    # # plt.savefig("SvsE " + ".pdf")
    # # files.download(exp_no+"_SvsE.png")
    # plt.show()

    # # Plot the average rewards with mean and standard deviation
    # plt.figure()
    # plt.plot(avg_step, label='Mean')
    # # plt.fill_between(range(step_size), trunc_avg_step + tunc_step_std, trunc_avg_step - tunc_step_std, alpha=0.5, label=' Std Dev')
    # plt.xlabel('Episode')
    # plt.ylabel('Cummulative Regret')
    # plt.title('Cummulative Average Regret vs Episode')
    # plt.legend()
    # # plt.savefig("SvsE " + ".pdf")
    # # files.download(exp_no+"_SvsE.png")
    # plt.show()


In [108]:
# # Average over 5 exeriments

# average_scores, cumm_regret = [], []
num_expts = 5
for i in range(num_expts):
    AvgOverExperiments()
    # print("Experiment: %d" % (i + 1))
    # # episode_average_scores, episode_cumm_regret =  wandb.agent(sweep_id, function=AvgOverExperiments, count=1)
    # wandb.agent(sweep_id, function=AvgOverExperiments, count=1)
    # # average_scores.append(episode_average_scores)
    # # cumm_regret.append(episode_cumm_regret)



/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 100	Average Score: 19.97
Episode 200	Average Score: 29.53
Episode 300	Average Score: 40.59
Episode 400	Average Score: 42.76
Episode 500	Average Score: 52.89
Episode 600	Average Score: 70.53
Episode 700	Average Score: 70.74
Episode 800	Average Score: 81.34
Episode 900	Average Score: 98.61
Episode 1000	Average Score: 107.87
Episode 1100	Average Score: 170.08
Environment CartPole-v1 solved in 1026 episodes!	Average Score: 195.02
0:01:55.448281
[195.02]


Episode 100	Average Score: 25.83
Episode 200	Average Score: 27.64
Episode 300	Average Score: 32.70
Episode 400	Average Score: 33.50
Episode 500	Average Score: 41.09
Episode 600	Average Score: 41.58
Episode 700	Average Score: 48.87
Episode 800	Average Score: 64.23
Episode 900	Average Score: 68.69
Episode 1000	Average Score: 104.74
Episode 1100	Average Score: 134.19
Episode 1200	Average Score: 170.33
Environment CartPole-v1 solved in 1183 episodes!	Average Score: 199.06
0:02:02.585020
[199.06]


Episode 100	Average Score: 25.79
Epis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_avg_curves(avg_reward, step_size=1):
    max_length = max(len(reward) for reward in avg_reward)

    # Initialize arrays to store sum and count of non-zero values for each index
    sum_non_zero = np.zeros(max_length)
    count_non_zero = np.zeros(max_length)

    # Iterate over each list in avg_reward
    for reward in avg_reward:
        for j, value in enumerate(reward):
            if j % step_size == 0:
                sum_non_zero[j] += value
                count_non_zero[j] += 1

    # Calculate mean and standard deviation using non-zero values
    mean_avg_reward = sum_non_zero / count_non_zero
    mean_reward_std = np.zeros(max_length)

    for reward in avg_reward:
        for j, value in enumerate(reward):
            if j % step_size == 0:
                mean_reward_std[j] += (value - mean_avg_reward[j]) ** 2

    mean_reward_std = np.sqrt(mean_reward_std / count_non_zero)

    # Plot the average rewards with mean and standard deviation
    plt.figure()
    plt.plot(mean_avg_reward, label='Mean')
    plt.fill_between(range(0, max_length, step_size), mean_avg_reward + mean_reward_std, mean_avg_reward - mean_reward_std, alpha=0.5, label=' Std Dev')
    plt.xlabel('Episode')
    plt.ylabel('Average Total Reward')
    plt.title('Average Total Reward vs Episode')
    plt.legend()
    plt.show()

# Example usage:
# average_scores = [[1, 2, 3], [4, 5, 6, 7], [8, 9]]

plot_avg_curves(np.array(average_scores), step_size=1)
